In [28]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [30]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [31]:
driver = Driver.driver_init()

In [32]:
product_paths = []

In [33]:
men = [2, 1, 2, 1, 1, 2, 1]
women = [2, 1, 1, 1, 1, 2, 1, 1]

url = 'https://arcteryx.co.kr/goods/catalog'    

In [34]:
## 남성
## 카테고리 분류 루프
for category_cnt in range(len(men)):
    pages_cnt = men[category_cnt]
    ## 각 옷 분류의 페이지 갯수 루프
    for page_num in range(pages_cnt):
        try:
            page = url + "?page=" + str(page_num+1) + '&category=00010001000' + str(category_cnt+1)
            driver.get(page)
            time.sleep(1)
        except:
            continue

        products = driver.find_elements('xpath', '//div[@id="searchedItemDisplay"]/ul/li')
        products_cnt = len(products)
        
        ## 각 product의 path를 product_paths 리스트에 저장
        for i in range(products_cnt):
            product = driver.find_elements('xpath', '//div[@class="searched_item_display"]/ul/li')[i]
            product_path = product.find_element('xpath', './div/div/a').get_attribute('href')
            product_paths.append(product_path)
    print(len(product_paths))

56
89
133
145
151
202
213


In [35]:
## 여성    
## 카테고리 분류 루프
for category_cnt in range(len(women)):
    pages_cnt = women[category_cnt]
    ## 각 옷 분류의 페이지 갯수 루프
    for page_num in range(pages_cnt):
        try:
            page = url + "?page=" + str(page_num+1) + '&category=00020001000' + str(category_cnt+1)
            driver.get(page)
            time.sleep(1)
        except:
            continue

        products = driver.find_elements('xpath', '//div[@id="searchedItemDisplay"]/ul/li')
        products_cnt = len(products)
        
        ## 각 product의 path를 product_paths 리스트에 저장
        for i in range(products_cnt):
            product = driver.find_elements('xpath', '//div[@class="searched_item_display"]/ul/li')[i]
            product_path = product.find_element('xpath', './div/div/a').get_attribute('href')
            product_paths.append(product_path)
    print(len(product_paths))

263
299
329
337
350
391
398
407
